In [2]:
import numpy as np
import open3d as o3d
import csv

In [3]:
lat = []
lon = []
alt = []
intensity = []

In [4]:
def LLA_to_XYZ(Lat,Lon,Alt,intensity):
    ''' Takes Latitude, Longitude and Altitdue and converts this data into
    x,y and z cartesion co-ordinates. Returns intensity as is.'''
    r_earth = (6378.137*1000)+Alt
    LatCos = np.cos(Lat*np.pi/180)
    LatSin = np.sin(Lat*np.pi/180)
    LonCos = np.cos(Lon*np.pi/180)
    LonSin = np.sin(Lon*np.pi/180)
    f = 1.0 / 298.257224
    C = 1.0/ np.sqrt(LatCos**2 + ((1-f)**2)*((LatSin)**2))
    S = ((1.0 - f)**2)*(C)
    x = (r_earth*C)*LatCos*LonCos
    y = (r_earth*C)*LatCos*LonSin
    z = (r_earth*S)*LatSin
    return x,y,z,intensity

In [5]:
def readfile(filename):
    ''' This function reads the file -"filename"- and appends 
    the data to the lat,lon,alt and intensity arrays '''
    global lat,lon,alt,intensity
    with open(filename) as csvfile:
        reader = csv.reader(csvfile, delimiter =' ')
        for row in reader:
            lat.append(float(row[0]))
            lon.append(float(row[1]))
            alt.append(float(row[2]))
            intensity.append(float(row[3]))

In [6]:
readfile("final_project_data/final_project_point_cloud.fuse")
points_xyzi = np.ones((len(lat),4))

In [7]:
for i in range(0,len(lat)):
    points_xyzi[i] = LLA_to_XYZ(lat[i],lon[i],alt[i],intensity[i])

In [8]:
print(points_xyzi.shape)

(430736, 4)


In [9]:
print(lat[31])
print(lon[31])
print(alt[31])
print(intensity[31])
print(points_xyzi[31])

45.90370042
11.02826286
226.2948
4.0
[4.36406352e+06 8.50522272e+05 4.55796734e+06 4.00000000e+00]


In [10]:
def write_point_cloud_object(array,name):
    ''' Takes array of point cloud (x,y,z) data and writes it to a 
    point cloud object file viewable in meshLab'''
    pc_object = open(name,"w")
    for j in array:
        pc_object.write("v "+str(j[0])+" "+str(j[1])+" "+str(j[2])+"\n")
    pc_object.close

In [11]:
write_point_cloud_object(points_xyzi,"All_data.obj")

In [12]:
#Using open3d - creating open3d object
p = o3d.geometry.PointCloud()
# print(points_xyzi.shape)
p.points = o3d.utility.Vector3dVector(points_xyzi[ : , 0:3])
# help(o3d.geometry.PointCloud().colors)
intens = np.zeros((np.size(points_xyzi[ : , 3]),3))
scaled_intensity = points_xyzi[ : , 3]/255
intens[:,0] = np.reshape(scaled_intensity,-1)
intens[:,1] = np.reshape(scaled_intensity,-1)
intens[:,2] = np.reshape(scaled_intensity,-1)
p.colors = o3d.utility.Vector3dVector(intens)
# # p.colors = o3d.utility.Vector3dVector(points_xyzi[ : , 3])
o3d.io.write_point_cloud("./Alldata.ply", p)
o3d.visualization.draw_geometries([p])

In [19]:
#Downsampling by voxel
down_voxel = o3d.geometry.voxel_down_sample(p, voxel_size=0.8)
o3d.io.write_point_cloud("./Downsampled_voxel.ply", down_voxel)
# o3d.visualization.draw_geometries([down_voxel])


#Downsampling uniformly
down_uniform = o3d.geometry.uniform_down_sample(p, every_k_points=5)
o3d.io.write_point_cloud("./Downsampled_uniform.ply", down_uniform)
# o3d.visualization.draw_geometries([down_uniform])

#Downsampling statistical outlier removal
down_stats, ind = o3d.geometry.statistical_outlier_removal(down_voxel,nb_neighbors=50,std_ratio=5.0)
# inliers = o3d.geometry.select_down_sample(down_stats, ind)
o3d.io.write_point_cloud("./Downsampled_Removed_Stat_Outliers.ply", down_stats)
o3d.visualization.draw_geometries([down_stats])
print(np.size(p))

1


In [15]:
print(np.size(p.points))
print(np.size(down_voxel.points))
print(np.size(down_stats.points))

1292208
53334
53088


In [18]:
#Get the normals
normals = down_stats
#Play with radius and nn numbers
o3d.geometry.estimate_normals(normals,search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1,max_nn=30)) 
o3d.visualization.draw_geometries([normals]) #press n to see the normals